In [2]:
import os, json
from dotenv import load_dotenv
from pymongo import MongoClient
from bson import json_util as bsonju

# --- parameter ---
BARCODE = "4809992000101"

# --- connect ---
load_dotenv()
client = MongoClient(os.getenv("MONGODB_URI"))
coll = client[os.getenv("MONGODB_DATABASE")]["inventories"]

# --- find the open form containing the barcode ---
doc = coll.find_one(
    {"is_empty": True, "items.barcode": BARCODE},
    {"_id": 0, "id": 1, "current_form_qr_code": 1, "is_empty": 1, "items": 1}
)

if not doc:
    raise RuntimeError("No open (is_empty=True) inventory contains that barcode.")

item = next((it for it in (doc.get("items") or []) if it.get("barcode") == BARCODE), None)
if not item:
    raise RuntimeError("Item not found in the matched inventory (unexpected).")

out = {

        "barcode": item.get("barcode"),
        "type": item.get("type"),
        "name": item.get("name"),
        "price": item.get("price"),
        "unit": item.get("unit"),
        "previous_quantity": item.get("previous_quantity"),
        "addstock": item.get("addstock"),
        "sold": item.get("sold"),
        "current_quantity": item.get("current_quantity"),
        "created": item.get("created"),
        "updated": item.get("updated"),
        "id": item.get("id"),
        "item_number": item.get("item_number"),
    
}

print(bsonju.dumps(out, ensure_ascii=False, indent=2))


{
  "barcode": "4809992000101",
  "type": "fuel",
  "name": "Premium Gasoline",
  "price": 69.5,
  "unit": "liter",
  "previous_quantity": 570,
  "addstock": 0,
  "sold": 0,
  "current_quantity": 0,
  "created": {
    "$date": 1760457682654
  },
  "updated": {
    "$date": 1760457682654
  },
  "id": "249f4956-70a7-4557-ad9a-95a400bb0f47",
  "item_number": "1"
}
